<a href="https://colab.research.google.com/github/GVSU-CIS635/projects-team-1-1/blob/main/logistic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing packages

In [2]:
# Installing Optuna first as it is an external dependency
!pip install optuna

# --- CONSOLIDATED IMPORTS ---
import pandas as pd
import numpy as np
import optuna
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import chi2_contingency

# SKLEARN MODULES
from sklearn.base import clone
from sklearn.model_selection import (
    train_test_split,
    GridSearchCV,
    StratifiedKFold,
    StratifiedShuffleSplit,
    cross_validate
)
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import (
    StandardScaler,
    OneHotEncoder,
    label_binarize # Although not used in final code, kept for completeness
)
from sklearn.impute import SimpleImputer

# SKLEARN MODELS
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

# SKLEARN METRICS
from sklearn.metrics import (
    accuracy_score,
    roc_auc_score,
    classification_report,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    precision_recall_curve, # Needed for threshold tuning
    roc_curve,
    auc
)

# Data Collection

In [ ]:
# Data Loading
df_train = pd.read_csv("https://raw.githubusercontent.com/GVSU-CIS635/projects-team-1-1/main/data/train.csv", sep=";", skipinitialspace=True)
df_test = pd.read_csv("https://raw.githubusercontent.com/GVSU-CIS635/projects-team-1-1/main/data/test.csv", sep=";", skipinitialspace=True)


# Missing Values

In [ ]:
# looking for missing data
print("Missing values found in Train\n", df_train.isnull().sum())

# looking for duplicate data
print("Duplicates found in Train ", df_train.duplicated().sum())

print("---------------------------------------------------------")
# looking for missing data
print("Missing values found in test\n", df_test.isnull().sum())

# looking for duplicate data
print("Duplicates found in test:", df_test.duplicated().sum())

Even though the source has already stated that the data has no missing values or duplicates, it is still good to verify this, since these steps affect all the later processes.

# Checking if Test is a subset of Train

In [ ]:
# Creating a boolean mask: for each test row, check if it appears in train
mask = df_test.merge(df_train.drop_duplicates(), how='left', indicator=True)['_merge'] == 'both'

In [ ]:
num_test = len(df_test)
num_matches = mask.sum()
num_missing = num_test - num_matches
percent_match = num_matches / num_test * 100

print(f"Test rows: {num_test}")
print(f"Rows that appear in train: {num_matches}")
print(f"Rows NOT found in train: {num_missing}")
print(f"Percent of test that is in train: {percent_match:.2f}%")

Since test is a subset of train we can use training data to split into train(70%) and test(30%) and use it for models.

# Feature Group Classification

In [ ]:
# Define feature groups
num_features = ["age", "balance", "day", "campaign", "pdays_numeric", "previous"]
cat_features = ["job", "marital", "education", "contact", "month", "poutcome"]
bin_features = ["default", "housing", "loan"]

# Data Cleaning

In [ ]:
def prepare_external_minimal(df_ext: pd.DataFrame) -> pd.DataFrame:

    # making a copy
    dfx = df_ext.copy()

    # Normalize headers
    dfx.columns = dfx.columns.str.strip().str.lower()

    if "duration" in dfx.columns:
        dfx = dfx.drop(columns=["duration"])

    # Fix mixed-type categorical columns
    cat_cols_train = df_train.select_dtypes(include=["object"]).columns.tolist()
    for c in cat_cols_train:
        df_train[c] = df_train[c].astype(str)

    dfx["contacted_before"] = (dfx["pdays"] != -1).astype(int)
    dfx["pdays_numeric"]    = dfx["pdays"].replace(-1, 0)
    for col in bin_features + ["y"]:
        if col in dfx.columns:
            dfx[col] = dfx[col].map({"yes": 1, "no": 0})
    return dfx

In [ ]:
df_train = prepare_external_minimal(df_train)
df_test  = prepare_external_minimal(df_test)

In [ ]:
# HL
gnb_ex_df_test = df_test.copy()

# Numeric Features Visualization

In [ ]:
# Box plot
plt.figure(figsize=(12,8))
for i, col in enumerate(num_features, 1):
    plt.subplot(3, 3, i)
    sns.boxplot(x = df_train[col], color="skyblue")
    plt.title(f"{col}")
plt.tight_layout()
plt.show()

In [ ]:
# Distribution
plt.figure(figsize=(12,8))
for i, col in enumerate(num_features, 1):
    plt.subplot(3, 3, i)
    sns.histplot(df_train[col], bins=30, kde=True)
    plt.title(f"{col}")
plt.tight_layout()
plt.show()

In [ ]:
# Correlation
corr = df_train[num_features].corr()
plt.figure(figsize=(8,6))
sns.heatmap(corr, annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Correlation Heatmap (Numeric Features)")
plt.show()


We are trying to understand the numerical features in this section:
- How are the features distributed? Do they show any skewness?
- What are the correlations among the features?
- This will help us make better judgments on whether remove outliers, normalize, or remove similar features.

From the box plot and the distribution graphs
- Age       : that looks ok, with outliers
- Balance   : most people fill in 0, maybe people over look this section
- Day       : no outliers, good distribution
- Duration  : dropped due to data leakage
- Campaign  : more than 6 calls during a campain and that would be too many call for one personm > skewed right
- Pdays = contacted_before + pdays_numeric bc -1 means client was not previously contacted, there are too many -1 (81.7%) > skewed right
- Previous  : most clients are new (81.7%) > skewed right

Notes:
- From the correlation heatmap, most features are not correlated to anothers so we can keep them all

Actions
- Age           : StandardScaler
- Balance       : Skewed right with negative values > StandardScaler
- Day           : StandardScaler
- Campaign      : log1p transformed > StandardScaler
- pdays_numeric : log1p transformed > StandardScaler
- previous      : log1p transformed > StandardScaler

What is log1p transform?
- Reduces skewness
- Makes the distribution more normal-like
- Helps with logistic regression

# Categorical features Visualization

In [ ]:
# Graphs
# Number of plots
n     = len(cat_features)         # 10
ncols = 3                         # 3 columns
nrows = n // ncols + 1            # 10 // 3 + 1 = 4

plt.figure(figsize=(5*ncols, 4*nrows))

for i, col in enumerate(cat_features, 1):
    plt.subplot(nrows, ncols, i)
    ax = sns.countplot(
        x=col,
        hue=col,
        data=df_train,
        palette='pastel',
        order=df_train[col].value_counts().index,
        legend=False
    )

    total = len(df_train)

    # find tallest bar to give extra y-axis space
    max_height = max(p.get_height() for p in ax.patches)
    ax.set_ylim(0, max_height * 1.10)  # 15% space above bars

    # annotate each bar
    for p in ax.patches:
        count = p.get_height()
        percentage = 100 * count / total

        # annotate text slightly above bar top
        ax.annotate(
            f'{percentage:.1f}%',
            xy=(p.get_x() + p.get_width() / 2., p.get_height()),
            xytext=(0, 6),                  # 6 points above the bar
            textcoords='offset points',
            ha='center', va='bottom',
            fontsize=9, color='black'
        )

    plt.title(col.capitalize(), fontsize=11)
    plt.xlabel('')
    plt.ylabel('')
    plt.xticks(rotation=45, ha='right')

plt.tight_layout()
plt.show()

From the bar charts:
- There are some dominant responses among the features; during training, the classes(freatures) will therefore be weighted.

What is Cramér’s V correlation coefficient?
- It measures the strength of association between two categorical variables.


In [ ]:
def cramers_v(x, y):
    table = pd.crosstab(x, y)
    chi2  = chi2_contingency(table, correction = False)[0]
    n     = table.sum().sum()
    k     = min(table.shape)
    return np.sqrt(chi2 / (n * (k - 1)))

# build the matrix
cramers = pd.DataFrame(index = cat_features, columns=cat_features, dtype=float)

for c1 in cat_features:
    for c2 in cat_features:
        cramers.loc[c1, c2] = cramers_v(df_train[c1], df_train[c2])

# visualize
plt.figure(figsize=(8,6))
sns.heatmap(cramers.astype(float), annot=True, cmap='Blues', fmt=".2f")
plt.title("Cramér's V Correlation between Categorical Features")
plt.tight_layout()
plt.show()


From the Cramer's V heat map
- There is no strong correlation among features
- 0.5 and 0.46 are the two most significant values
- education and job are kind of related - if you are in school > you are a student
- month and housing - in the summer, people are just happy to go buy a house
- month and contact - in the summer, people are just happy to pickup the phone

# Train/Test split

In [ ]:
# 70% train / 30% internal test (keep separate test.csv untouched)
X = df_train[num_features + cat_features + bin_features]
y = df_train["y"].astype(int)

In [ ]:
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=42)

for train_idx, test_idx in sss.split(X, y):
  X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
  y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

In [ ]:
# HL
gnb_X_train = X_train.copy()
gnb_X_test  = X_train.copy()
gnb_y_train = y_train.copy()
gnb_y_test  = y_train.copy()

# Preprocessing

In [ ]:
# Imputer + Scaler are placed in every pipeline to avoid leakage and for consistent processing.
imputer = SimpleImputer(strategy='median')
scaler  = StandardScaler()

In [ ]:
numeric_transformer = Pipeline(
  steps=[
  ("imputer", SimpleImputer(strategy="median")),
  ("scaler", StandardScaler())
])

categorical_transformer = Pipeline(
  steps=[
  ("imputer", SimpleImputer(strategy="most_frequent")),
  ("encoder", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
])

In [ ]:
# Preprocessing transformer
preprocessor = ColumnTransformer(
  transformers=[
    ('num', numeric_transformer, num_features),
    ('cat', categorical_transformer, cat_features),
    ('bin', 'passthrough', bin_features),
  ],
)

# HL
preprocessor_gnb = ColumnTransformer(
  transformers=[
    ('num', numeric_transformer, num_features),
    ('cat', categorical_transformer, cat_features),
    ('bin', 'passthrough', bin_features),
  ],
)

### After the proprocessing, everyone can do what ever they want

In [ ]:
# train.csv > train_set + test_set (70/30) use shuffle

# 1. internal testing for the test_set

# 2. external testing using test.csv

# Compare 1 vs 2

# Sri - Logistic Regression
# San - Random Forest

## Pipeline

In [ ]:
# creating pipelines
lr_pipe = Pipeline([
  ('preprocess', preprocessor),
  ('clf', LogisticRegression(max_iter=2000))
])

## Parameter Grid

In [ ]:
# Parameters for Logistic Regression
lr_param_grid = {
  'clf__C': [0.01, 0.1, 1, 10, 100],
}


# Optional
# print("\n[Hyperparameter Grid]")
# for param, values in rf_param_grid.items():
#     print(f"  {param}: {values}")
# print(f"Total combinations: {len(rf_param_grid['clf__n_estimators']) * len(rf_param_grid['clf__max_depth']) * len(rf_param_grid['clf__min_samples_split'])}")


## StratifiedKFold

In [ ]:
# creating 2 seperate because this avoids optimistic bias because you evaluate on folds that the model has not seen during hyperparameter tuning.
# use the different random_state, otherwise the folds will still be identical.

# creating a StratifiedKFold classifier to train the models
cv_tune = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# creating a StratifiedKFold classifier for Cross Validation
cv_eval = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)

## GridSearchCV

In [ ]:
gs_lr = GridSearchCV(lr_pipe, lr_param_grid, cv = cv_tune, scoring='f1_weighted', n_jobs=-1)


## Fit Models

In [ ]:
# Logistic Regression model Fit
gs_lr.fit(X_train, y_train)

## Best Estimator

In [ ]:
best_lr = gs_lr.best_estimator_ # Best Logistic Regression Estimator

## Cross Validation

In [ ]:
def run_cv(model_name, model, X_train, y_train, cv, scoring = None, return_train_score=False):
  if scoring is None:
    scoring = ["accuracy", "f1_weighted", "roc_auc_ovr_weighted"]
  scores = cross_validate(model, X_train, y_train, cv=cv_eval, scoring=scoring, return_train_score=False)
  return ({
      "Model": model_name,
      "Accuracy Mean ± Std": f'{np.mean(scores["test_accuracy"]):.4f} ± {np.std(scores["test_accuracy"], ddof=1):.4f}',
      "F1 Mean  ± Std": f'{np.mean(scores["test_f1_weighted"]):.4f} ± {np.std(scores["test_f1_weighted"], ddof=1):.4f}',
      "AUC-ROC Mean  ± Std": f'{np.mean(scores["test_roc_auc_ovr_weighted"]):.4f} ± {np.std(scores["test_roc_auc_ovr_weighted"], ddof=1):.4f}'
  })

In [ ]:
scores_lr = run_cv("Logistic", best_lr, X_train, y_train, cv=cv_eval)

print("----------------- Logistic Regression ---------------------------")
print(pd.DataFrame([scores_lr]))

## Prediction

In [ ]:
# Predictions
y_pred_lr = best_lr.predict(X_test)
y_proba_lr = best_lr.predict_proba(X_test)


## Accuracy, F1_score, roc_auc_score

In [ ]:
def print_acc_f1_roc(y_test, y_pred, y_proba):
  # Accuracy
  internal_accuracy = accuracy_score(y_test, y_pred)

  # F1 Score (weighted for multiclass safety)
  internal_f1 = f1_score(y_test, y_pred, average='weighted', pos_label=1)

  # ROC-AUC binary
  internal_auc = roc_auc_score(y_test, y_proba[:, 1])

  # Confusion Matrix
  cm = confusion_matrix(y_test, y_pred)

  # Classification Report
  report = classification_report(y_test, y_pred)

  print(f"Test Accuracy: {internal_accuracy:.4f}")
  print(f"Test F1 Score: {internal_f1:.4f}")
  print(f"Test ROC-AUC: {internal_auc:.4f}\n")

  print("\nClassification report:\n", report)

  print("Confusion Matrix:\n", cm)

  # return internal_accuracy, internal_f1, internal_auc


In [ ]:
print_acc_f1_roc(y_test, y_pred_lr, y_proba_lr)


## EXTERNAL TEST

In [ ]:
X_test = df_test[num_features + cat_features + bin_features]
y_test = df_test['y'].astype(int)

In [ ]:
# Predictions
y_pred_lr = best_lr.predict(X_test)
y_proba_lr = best_lr.predict_proba(X_test)


In [ ]:
print_acc_f1_roc(y_test, y_pred_lr, y_proba_lr)

## Plotting ROC_Curve

In [ ]:
# Dictionary of models
models = {
    "Logistic Regression": best_lr
}

In [ ]:
for name, model in models.items():
    y_true_all = []
    y_prob_all = []

    for train_idx, test_idx in cv_eval.split(X, y):
        est = clone(model)
        est.fit(X.iloc[train_idx], y.iloc[train_idx])

        # Predict probabilities for class 1
        if hasattr(est, "predict_proba"):
            y_prob = est.predict_proba(X.iloc[test_idx])[:, 1]
        elif hasattr(est, "decision_function"):
            y_prob = est.decision_function(X.iloc[test_idx])
        else:
            y_prob = est.predict(X.iloc[test_idx])  # fallback

        y_true_all.append(y.iloc[test_idx])
        y_prob_all.append(y_prob)

    # Concatenate results from all folds
    y_true_concat = np.concatenate(y_true_all)
    y_prob_concat = np.concatenate(y_prob_all)

    fpr, tpr, _ = roc_curve(y_true_concat, y_prob_concat)
    roc_auc = auc(fpr, tpr)

    plt.plot(fpr, tpr, lw=2, label=f"{name} (AUC = {roc_auc:.3f})")


plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves for All Models')
plt.legend(loc='lower right')
plt.grid(True)
plt.tight_layout()
plt.show()